# Chapter 4: Property-Based Testing

In Chapters 1–3, we moved from a quick, monolithic solver to a modular design with unit tests.
Unit tests gave us confidence, but only for cases that we handpicked.

Now, we’ll take the next step by introducing property-based testing via the Hypothesis
library:

 - Instead of manually writing individual test cases, we’ll state general properties such as conservation, symmetry, maximum principle intuition, or linearity.
 - Hypothesis will generate many random inputs automatically, exploring cases we might never think to test.
 - When a property fails, Hypothesis will shrink the failing case to the simplest possible counterexample, the testing equivalent of a controlled scientific experiment.

## What is a "property"?

A property is a statement about your program that you expect to hold for all valid inputs.
It’s a general rule, not just a single example.

For our 1-D heat equation solver, some key properties might include:

- **Conservation under insulated BCs:** total heat stays constant.
- **Boundary work balance:** with fixed flux BCs, the heat change matches net boundary flux over Δt.
- **Symmetry:** symmetric initial states stay symmetric under symmetric BCs.
- **Max Principle:** Diffusion shouldn’t create new temperature extremes.
- **Linearity:** The discrete update should scale linearly with the initial temperature distribution.

## Unit tests vs. property-based tests:
 - Unit tests: "for this input, expect this output"
 - Property tests: "for all inputs satisfying these preconditions, this relation should hold"

Property tests complement unit tests:
 - Unit tests are concrete and focused.
 - Property tests are broader and can reveal edge cases you never anticipated.

## How this relates to Chapter 2

In Chapter 2, we introduced preconditions, postconditions, and invariants as lightweight specifications for our solver.

Property-based testing is a natural extension of these ideas:

 - The precondition {P} now defines the input space that Hypothesis will explore.
 - The postcondition {Q} or invariant becomes the property being checked.
 - When a test fails, Hypothesis provides a counterexample, helping you determine whether:
   - The precondition was too weak (i.e., it allowed invalid inputs).
   - The postcondition was too strong (i.e., it ruled out valid outputs).
   - There’s a bug in the implementation.

Think of this as turning your Chapter 2 contracts into scientific hypotheses:

 - "If the precondition holds, the property should always be true."

Hypothesis plays the role of the experimenter, probing your code with many randomized “experiments” to try and refute your claim.

## The `Hypothesis` library

Hypothesis is a Python library for property-based testing.
Instead of handpicking test cases, you describe the space of valid inputs, and it generates random examples within that space.

If a failure is found, Hypothesis automatically shrinks the input to the simplest version that still causes the failure. This helps you debug by giving a clear, minimal failing example.

Let's revisit our `div` function from the previous chapters.

In [3]:
def div(x, y):
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

In Chapter 3, we tested this manually:

In [4]:
def test_division():
    div(7, 25)

test_division()

AssertionError: 

Running this fails because 7 / 25 cannot be represented exactly in floating point.
But notice: we only found this case because we happened to test exactly this input.
If we had tested other pairs like the ones listed below, the test
might have passed, which would leave us unaware of the issue.

In [5]:
def test_more_divisions():
    div(7, 26)
    div(7, 24)
    div(7, 23)
    div(6, 1)
    div(2, 4)
    div(0, 1)
    div(46, 7)
    div(7657, 26)
    div(1, 3)
    div (23, 23424)
    div(1000, 3)
    div(123,456)
    print("No assertion violations encountered!")

test_more_divisions()

No assertion violations encountered!


### Hypothesis to the rescue

Let's use Hypothesis to generate a wide range of test cases for our division function.

In [7]:
from hypothesis import assume, given, strategies as st

@given(st.integers(), st.integers())
def test_div_property(x, y):
    assume(y != 0)
    div(x, y)

The above test definition highlights the key components of property-based testing with Hypothesis:

1. The `@given` decorator
    - This is the main entry point for defining a property-based test.
    - It takes zero or more strategies as arguments, which describe how Hypothesis should generate input data for the test.

2. Strategies
    - Strategies define the space of possible inputs.
    - In our example, we used `st.integers()` to generate random integers for both x and y.

We also used the `assume` function to enforce our precondition {P}.
Here, the precondition is that `y != 0` to avoid division by zero. Adding `assume(y != 0)` ensures that we only test valid inputs.

With these pieces in place, we can now run the property-based test and let Hypothesis explore a wide range of inputs automatically:

In [8]:
test_div_property()

AssertionError: 

Running the above property-based test should reveal a pair of integers (x, y) that violate the division property. Recall, the inputs are generated randomly, so you might see different failing cases each time you run the test. 

### Shrinking

Hypothesis automatically *shrinks* the input, step by step, until it finds the simplest failing 
example. This makes debugging much easier, since you don’t need to wade through massive random inputs.

### Combining strategies

In the above example, we only tested integer division. However, we can also test floating-point division by using the `st.floats()` strategy from Hypothesis. Better yet, we can combine both strategies to test the `div` function with a wider range of inputs:

In [9]:
from hypothesis import assume, given, strategies as st

# Combine integers and floats into a single strategy:
numbers = st.one_of(st.integers(), st.floats(allow_nan=False, allow_infinity=False))

@given(numbers, numbers)
def test_div_property(x, y):
    assume(y != 0)
    div(x, y)

In [10]:
test_div_property()

AssertionError: 

### Controlling how many examples are generated

By default, `hypothesis` generates 100 random input values. You can control this with the @settings decorator:

In [72]:
from hypothesis import given, settings, strategies as st

ctr = 0

@settings(max_examples=10)
@given(st.integers())
def test_random(x):
    global ctr; ctr += 1
    print(f"Test no. {ctr} with random input {x}")

test_random()

Test no. 1 with random input 0
Test no. 2 with random input -8477684967185004649
Test no. 3 with random input 6386
Test no. 4 with random input -31712
Test no. 5 with random input -17165
Test no. 6 with random input 4
Test no. 7 with random input 114
Test no. 8 with random input -24931
Test no. 9 with random input -1854951769
Test no. 10 with random input -16623


In the above cell, we reduced `max_examples` from the default 100 to 10. In practice, one
should consider increasing the `max_examples` parameter based on the complexity of the property being tested, particularly in the absence of failing cases when no counterexamples are found.

## Back to the 1-D Heat Equation Solver

Now let's use Hypothesis to test our heat equation solver. Execute the below cell that contains the entire source code of our solver, so it becomes available for property-based testing.

In [34]:
from dataclasses import dataclass
from numpy import isclose
from pytest import approx

# Data abstractions

@dataclass
class Mesh:
    """Uniform 1-D mesh."""
    dx: float
    N: int  # number of cells
    def cell_centered_arr(self): return [0.0] * self.N
    def face_centered_arr(self): return [0.0] * (self.N + 1)

@dataclass
class BC:
    """Neumann flux boundary conditions on the diffusive flux."""
    qL: float
    qR: float

# Behavior abstractions

def apply_bc(F: list, qL: float, qR: float) -> None:
    """Apply Neumann BCs to face fluxes F."""
    assert len(F) > 1
    F[0], F[-1] = qL, qR

def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update face fluxes F based on cell field u."""

    assert dx > 0, "Invalid grid spacing"
    assert len(F) == len(u) + 1, "Inconsistent face flux array length"

    for i in range(1, len(u)):
        F[i] = -kappa * (u[i] - u[i-1]) / dx

def divergence_telescoping(F, dudt, dx):
    """Check the finite volume telescoping property."""
    total_divergence = sum(dudt) * dx
    boundary_flux = F[0] - F[-1]
    return isclose(total_divergence, boundary_flux)

def divergence(dudt: list, F: list, dx: float) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    assert len(F) == len(dudt) + 1
    for i in range(len(dudt)):
        dudt[i] = (F[i] - F[i+1]) / dx
    assert divergence_telescoping(F, dudt, dx)

def step(u: list, F: list, dudt: list, kappa: float, dt: float, mesh: Mesh, bc: BC) -> list:
    """Advance cell field u by one time step using explicit Euler method."""
    assert dt > 0

    apply_bc(F, bc.qL, bc.qR)
    flux(F, u, kappa, mesh.dx)
    divergence(dudt, F, mesh.dx)

    for i in range(mesh.N):
        u[i] += dt * dudt[i]

def Euler_FTCS_stability(dt: float, dx: float, kappa: float) -> bool:
    """Check stability condition for explicit Euler (FTCS) scheme."""
    r = kappa * dt / dx**2
    return r <= 0.5

def heat_is_conserved(u_sum_before: float, u_sum_after: float, dt: float, bc: BC, mesh: Mesh) -> bool:
    """Check if heat is conserved."""
    expected_change = dt * (bc.qL - bc.qR)
    actual_change = u_sum_after - u_sum_before
    return isclose(actual_change, expected_change)

def solve(u0: list, kappa: float, dt: float, nt: int, dx: float, bc: BC) -> list:
    """Orchestrate nt steps over cell field u."""

    assert nt > 0
    assert Euler_FTCS_stability(dt, dx, kappa), "Stability condition not met"

    mesh = Mesh(dx, N=len(u0))
    u = u0.copy()
    F = mesh.face_centered_arr()
    dudt = mesh.cell_centered_arr()

    for _ in range(nt):
        u_sum_before = sum(u) * mesh.dx
        step(u, F, dudt, kappa, dt, mesh, bc)
        u_sum_after = sum(u) * mesh.dx
        assert heat_is_conserved(u_sum_before, u_sum_after, dt, bc, mesh), "Heat not conserved"

    return u



### Symmetry

We can test the symmetry property of the heat equation solver by checking if the solution remains symmetric when the initial conditions are mirrored and boundary conditions are applied symmetrically. Below is an example of how to implement this test using Hypothesis:


In [36]:
# Customized float strategies with min and max values for various solver parameters
floats_sane = st.floats(min_value=-1e3, max_value=1e3, allow_nan=False, allow_infinity=False)
dx_strat     = st.floats(min_value=1e-3, max_value=10, allow_nan=False, allow_infinity=False)
kappa_strat  = st.floats(min_value=1e-4, max_value=10, allow_nan=False, allow_infinity=False)
dt_strat     = st.floats(min_value=1e-6, max_value=10, allow_nan=False, allow_infinity=False)

def symmetric_field(min_n=3, max_n=31):
    # Produce palindromic arrays (odd length preferred for strict central symmetry)
    # Build half then mirror
    half = st.lists(floats_sane, min_size=(min_n//2), max_size=(max_n//2))
    center = floats_sane
    return st.builds(lambda h, c: h + [c] + list(reversed(h)), half, center)

@given(symmetric_field(), kappa_strat, dx_strat, dt_strat)
def test_symmetry_preserved_under_symmetric_BCs(u_sym, kappa, dx, dt):
    """
    Symmetric initial state stays symmetric for one step if qL==qR and r<=1/2.
    """
    assume(len(u_sym) >= 3 and len(u_sym) % 2 == 1)
    r = kappa*dt/(dx*dx)
    assume(r <= 0.5)

    mesh = Mesh(dx=dx, N=len(u_sym))
    bc   = BC(qL=0.0, qR=0.0)

    F    = mesh.face_centered_arr()
    dudt = mesh.cell_centered_arr()
    u    = u_sym[:]

    step(u, F, dudt, kappa=kappa, dt=dt, mesh=mesh, bc=bc)
    assert u == approx(list(reversed(u)))

In [38]:
test_symmetry_preserved_under_symmetric_BCs()
print("Test completed")

Test completed



### Conservation

We can test the conservation property of the heat equation solver by checking if the total heat remains constant over time when insulated boundary conditions are applied. Below is an example of how to implement this test using Hypothesis:


In [40]:
# A custom strategy for boundary condition fluxes
bc_flux_strat= st.floats(min_value=-100, max_value=100, allow_nan=False, allow_infinity=False)

def cell_field(min_n=3, max_n=30):
    return st.lists(floats_sane, min_size=min_n, max_size=max_n)

# Stability guard: r = kappa*dt/dx^2 <= 0.5
def stable_tuple():
    return st.tuples(cell_field(),
                     kappa_strat,
                     dx_strat,
                     dt_strat).filter(lambda t: (t[1]*t[3]/(t[2]*t[2])) <= 0.5)



@given(stable_tuple(), bc_flux_strat, bc_flux_strat)
def test_conservation_insulated_is_invariant(args, qL, qR):
    """
    With qL=qR=0, the discrete heat sum(u)*dx must remain invariant after one step.
    """
    u0, kappa, dx, dt = args
    assume(len(u0) >= 3)
    mesh = Mesh(dx=dx, N=len(u0))
    bc   = BC(qL=0.0, qR=0.0)

    # Prepare state arrays
    F    = mesh.face_centered_arr()
    dudt = mesh.cell_centered_arr()
    u    = u0[:]  # copy

    total_before = sum(u) * dx
    step(u, F, dudt, kappa=kappa, dt=dt, mesh=mesh, bc=bc)
    total_after  = sum(u) * dx

    assert total_after == approx(total_before, rel=1e-12, abs=1e-12)

In [41]:
test_conservation_insulated_is_invariant()
print("Test completed")

Test completed



### More properties

[TODO: add more property checks...]

## What we just did

Property-based testing lets you codify physics and math as executable claims:

 - We encoded conservation, boundary work, symmetry, a max-principle intuition, telescoping, linearity.

 - Hypothesis searched broad input spaces and shrank counterexamples when things went wrong.

 - This directly operationalizes the scientific method for software: state a hypothesis (property), attempt refutation (generation + shrinking), refine code or specs.

## Looking Ahead

In Chapter 5, we’ll step across the next rung of the rigor ladder: theorem proving—automating reasoning over all inputs within a finite domain and turning some of these properties into machine-checked contracts.


---

*This notebook is a part of the "Rigor and Reasoning in Research Software" (R3Sw) tutorial, led by Alper Altuntas and sponsored by the Better Scientific Software (BSSw) Fellowship Program. Copyright © 2025*